# feature extraction

In [755]:
import os
import gc
import sys
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
from collections import defaultdict
from multiprocessing import cpu_count, Pool

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

import warnings
warnings.simplefilter('ignore')

In [756]:
PATH = os.path.join('..', 'remove_outlier_data')

KEY = 'card_id'
SEED = 6
NTHREAD = cpu_count()
NFOLD = 5

params = {
    'num_leaves': 31,
    'min_data_in_leaf': 30, 
    'objective':'regression',
    'max_depth': -1,
    'learning_rate': 0.01,
    'boosting': 'gbdt',
    'feature_fraction': 0.6,
    'bagging_freq': 1,
    'bagging_fraction': 0.8,
    'bagging_seed': 11,
    'metric': 'rmse',
    'lambda_l1': 0.1,
    'verbosity': -1,
    'nthread': NTHREAD,
    'random_state': SEED
}

features = []

features += [f'f10{i}.pkl' for i in (2, 3, 4, 5, 6, 7, 8)]
features += [f'f11{i}_{j}.pkl' for i in (1,) 
                               for j in ('Y', 'N')]
features += [f'f13{i}.pkl' for i in (1, 3, 4, 5)]

features += [f'f20{i}.pkl' for i in (2, 3, 4, 5, 6, 7, 8)]
features += [f'f23{i}.pkl' for i in (1, 5)]

features += [f'f30{i}.pkl' for i in (2, )]

In [757]:
train = pd.read_csv(os.path.join(PATH, 'train.csv'))
test = pd.read_csv(os.path.join(PATH, 'test.csv'))

for f in tqdm(features):
#     print(f)
    t = pd.read_pickle(os.path.join('..', 'remove_outlier_feature', f))
    train = pd.merge(train, t, on=KEY, how='left')
    test = pd.merge(test, t, on=KEY, how='left')

100%|██████████| 22/22 [00:28<00:00,  1.30s/it]


In [758]:
df = pd.concat([train, test], axis=0)
df['first_active_month'] = pd.to_datetime(df['first_active_month'])

date_features = [
    'hist_purchase_date_max','hist_purchase_date_min',
    'new_purchase_date_max', 'new_purchase_date_min',
    'Y_hist_auth_purchase_date_max', 'N_hist_auth_purchase_date_max',
    'Y_hist_auth_purchase_date_min', 'N_hist_auth_purchase_date_min'
]

for f in date_features:
    df[f] = pd.to_datetime(df[f])

df['hist_first_buy'] = (df['hist_purchase_date_min'].dt.date - df['first_active_month'].dt.date).dt.days
df['hist_last_buy'] = (df['hist_purchase_date_max'].dt.date - df['first_active_month'].dt.date).dt.days
df['new_first_buy'] = (df['new_purchase_date_min'].dt.date - df['first_active_month'].dt.date).dt.days
df['new_last_buy'] = (df['new_purchase_date_max'].dt.date - df['first_active_month'].dt.date).dt.days

for f in date_features:
    df[f] = df[f].astype(np.int64) * 1e-9

df['card_id_cnt_total'] = df['new_card_id_count'] + df['hist_card_id_count']
df['purchase_amount_total'] = df['new_purchase_amount_sum'] + df['hist_purchase_amount_sum']
df['purchase_amount_mean'] = df['new_purchase_amount_mean'] + df['hist_purchase_amount_mean']
df['purchase_amount_max'] = df['new_purchase_amount_max'] + df['hist_purchase_amount_max']
df['purchase_amount_min'] = df['new_purchase_amount_min'] + df['hist_purchase_amount_min']
df['sum_new_CLV'] = df['new_card_id_count'] * df['new_purchase_amount_sum'] / df['new_month_diff_mean']
df['sum_hist_CLV'] = df['hist_card_id_count'] * df['hist_purchase_amount_sum'] / df['hist_month_diff_mean']
df['sum_CLV_ratio'] = df['sum_new_CLV'] / df['sum_hist_CLV']

# df['outliers_1'] = df['hist_month_nunique'].apply(lambda x: np.where(x > 3, 1, 0))
# df['outliers_2'] = df['hist_month_diff_min'].apply(lambda x: np.where(x < 13, 1, 0))
# df['outliers_3'] = df['hist_month_diff_max'].apply(lambda x: np.where(x < 14, 1, 0))
# df['outliers_4'] = df['hist_month_lag_max'].apply(lambda x: np.where(x > -7, 1, 0))
# df['outliers_5'] = df['hist_month_lag_min'].apply(lambda x: np.where(x < -2, 1, 0))

# df['outliers_sum'] = df[['outliers_1', 'outliers_2', 'outliers_3', 'outliers_4', 'outliers_5']].apply(np.sum, axis=1)

categorical_features = [
    'feature_1', 'feature_2', 'feature_3',
]

# df['hist_subsector_id_nunique_x_hist_merchant_id_nunique '] = df['hist_subsector_id_nunique'] / df['hist_merchant_id_nunique']
# df['hist_merchant_id_nunique_x_hist_merchant_category_id_nunique'] = df['hist_merchant_id_nunique'] / df['hist_merchant_category_id_nunique']
# df['new_subsector_id_nunique_x_new_merchant_id_nunique '] = df['new_subsector_id_nunique'] / df['new_merchant_id_nunique']
# df['new_merchant_id_nunique_x_new_merchant_category_id_nunique'] = df['new_merchant_id_nunique'] / df['new_merchant_category_id_nunique']


for c in categorical_features:
    df[c] = df[c].astype('category')

df['nans'] = df.isnull().sum(axis=1)

train = df[df['target'].notnull()]
test = df[df['target'].isnull()]
y = train['target']

del df
gc.collect()

3196

# PCA

In [759]:
col_not_to_use = []

In [760]:
from sklearn.decomposition import PCA

In [761]:
categorical_features = ['feature_1', 'feature_2', 'feature_3']
pca = PCA(n_components=1)
pca.fit(train[categorical_features])
pca_train_values = pca.transform(train[categorical_features])
pca_test_values = pca.transform(test[categorical_features])

# col_not_to_use += ['feature_1', 'feature_2', 'feature_3']

pca_train_values = np.transpose(pca_train_values, (1, 0))
pca_test_values = np.transpose(pca_test_values, (1, 0))

for e, (pca_train, pca_test) in enumerate(zip(pca_train_values, pca_test_values)):
    train[f'pca_feature_{e}'] = pca_train
    test[f'pca_feature_{e}'] = pca_test

In [762]:
# features = ['feature_1_outliers_mean', 'feature_2_outliers_mean', 'feature_3_outliers_mean']
# pca = PCA(n_components=1)
# pca.fit(train[features])
# pca_train_values = pca.transform(train[features])
# pca_test_values = pca.transform(test[features])

# col_not_to_use += ['feature_1_outliers_mean', 'feature_2_outliers_mean', 'feature_3_outliers_mean']

# pca_train_values = np.transpose(pca_train_values, (1, 0))
# pca_test_values = np.transpose(pca_test_values, (1, 0))

# for e, (pca_train, pca_test) in enumerate(zip(pca_train_values, pca_test_values)):
#     train[f'pca_feature_outliers_mean_{e}'] = pca_train
#     test[f'pca_feature_outliers_mean_{e}'] = pca_test

In [763]:
# features = ['feature_1_outliers_sum', 'feature_2_outliers_sum', 'feature_3_outliers_sum']
# pca = PCA(n_components=1)
# pca.fit(train[features])
# pca_train_values = pca.transform(train[features])
# pca_test_values = pca.transform(test[features])

# col_not_to_use += ['feature_1_outliers_sum', 'feature_2_outliers_sum', 'feature_3_outliers_sum']

# pca_train_values = np.transpose(pca_train_values, (1, 0))
# pca_test_values = np.transpose(pca_test_values, (1, 0))

# for e, (pca_train, pca_test) in enumerate(zip(pca_train_values, pca_test_values)):
#     train[f'pca_feature_outliers_sum_{e}'] = pca_train
#     test[f'pca_feature_outliers_sum_{e}'] = pca_test

# exclude extra columns

In [764]:
col_not_to_use += [
    'first_active_month', 'card_id', 'target', 'outliers',
]

# col_not_to_use += [c for c in train.columns if ('duration' in c) or ('amount_month_ratio' in c) or ('skew' in c)]
col_to_use = [c for c in train.columns if c not in col_not_to_use]

gc.collect()

X_temp = train[col_to_use]
X_temp_test = test[col_to_use]

In [765]:
X_temp.shape

(201917, 625)

# CV

In [767]:
columns = X_temp.columns

In [768]:
columns = [c for c in columns if c not in use_columns]

In [769]:
folds = KFold(n_splits=4, shuffle=True, random_state=SEED)

pre_score = 10000
# use_columns = []

for c in columns:
    X = X_temp[use_columns + [c]]
    X_test = X_temp_test[use_columns + [c]]
    scores = []

    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        dtrain = lgb.Dataset(X.iloc[train_index], label=y.iloc[train_index])
        dvalid = lgb.Dataset(X.iloc[valid_index], label=y.iloc[valid_index])

        params = {
            'boosting': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': 0.01,
            'subsample': 0.9855232997390695,
            'max_depth': 7,
            'top_rate': 0.9064148448434349,
            'num_leaves': 63,
            'min_child_weight': 41.9612869171337,
            'other_rate': 0.0721768246018207,
            'reg_alpha': 9.677537745007898,
            'colsample_bytree': 0.5665320670155495,
            'min_split_gain': 9.820197773625843,
            'reg_lambda': 8.2532317400459,
            'min_data_in_leaf': 21,
            'verbose': -1,
            'seed':int(2**fold_n),
            'bagging_seed':int(2**fold_n),
            'drop_seed':int(2**fold_n)
        }

        model = lgb.train(
            params,
            dtrain,
            2000,          
            valid_sets=[dtrain, dvalid],
            verbose_eval=-1,
            early_stopping_rounds=200)

        y_pred_valid = model.predict(X.iloc[valid_index], num_iteration=model.best_iteration)
        scores.append(np.sqrt(mean_squared_error(y.iloc[valid_index], y_pred_valid)))

        del model
    
    if np.mean(scores) < pre_score:
        pre_score = np.mean(scores)
        use_columns.append(c)
    
    print('shape:', X.shape)
    print('CV {0:} mean score: {1:.4f}, std: {2:.4f}, max: {3:.4f}, min: {4:.4f}.'.format(4, np.mean(scores), np.std(scores), np.max(scores), np.min(scores)))

Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1630]	training's rmse: 3.29243	valid_1's rmse: 3.54939
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[796]	training's rmse: 3.40477	valid_1's rmse: 3.65362
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1076]	training's rmse: 3.33448	valid_1's rmse: 3.71171
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[984]	training's rmse: 3.37111	valid_1's rmse: 3.65146
shape: (201917, 104)
CV 4 mean score: 3.6415, std: 0.0584, max: 3.7117, min: 3.5494.
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1291]	training's rmse: 3.34204	valid_1's rmse: 3.54789
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[779]	training's rmse: 3.40653	valid_1's rmse: 3.

Early stopping, best iteration is:
[829]	training's rmse: 3.39934	valid_1's rmse: 3.65002
shape: (201917, 107)
CV 4 mean score: 3.6406, std: 0.0590, max: 3.7118, min: 3.5479.
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1260]	training's rmse: 3.34802	valid_1's rmse: 3.54787
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[836]	training's rmse: 3.39719	valid_1's rmse: 3.65346
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[873]	training's rmse: 3.36962	valid_1's rmse: 3.71149
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1001]	training's rmse: 3.3694	valid_1's rmse: 3.65054
shape: (201917, 107)
CV 4 mean score: 3.6408, std: 0.0589, max: 3.7115, min: 3.5479.
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[877]	training's rmse: 3.4158

Early stopping, best iteration is:
[1004]	training's rmse: 3.34298	valid_1's rmse: 3.71388
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1114]	training's rmse: 3.35013	valid_1's rmse: 3.65167
shape: (201917, 107)
CV 4 mean score: 3.6416, std: 0.0597, max: 3.7139, min: 3.5477.
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1131]	training's rmse: 3.37098	valid_1's rmse: 3.54814
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[765]	training's rmse: 3.41074	valid_1's rmse: 3.6523
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[839]	training's rmse: 3.37511	valid_1's rmse: 3.71326
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[849]	training's rmse: 3.39532	valid_1's rmse: 3.65132
shape: (201917, 107)
CV 4 mean score: 3.6413, std: 0.0593,

Early stopping, best iteration is:
[791]	training's rmse: 3.40616	valid_1's rmse: 3.65232
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[937]	training's rmse: 3.3565	valid_1's rmse: 3.71219
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1111]	training's rmse: 3.34993	valid_1's rmse: 3.65121
shape: (201917, 107)
CV 4 mean score: 3.6411, std: 0.0588, max: 3.7122, min: 3.5487.
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1338]	training's rmse: 3.33458	valid_1's rmse: 3.54786
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[790]	training's rmse: 3.40569	valid_1's rmse: 3.65371
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1046]	training's rmse: 3.34043	valid_1's rmse: 3.71165
Training until validation scores don't improve for 200 rou

Early stopping, best iteration is:
[1235]	training's rmse: 3.3499	valid_1's rmse: 3.54837
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[739]	training's rmse: 3.41374	valid_1's rmse: 3.65074
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[870]	training's rmse: 3.36775	valid_1's rmse: 3.71356
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[828]	training's rmse: 3.39672	valid_1's rmse: 3.65041
shape: (201917, 108)
CV 4 mean score: 3.6408, std: 0.0592, max: 3.7136, min: 3.5484.
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1276]	training's rmse: 3.342	valid_1's rmse: 3.54822
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[786]	training's rmse: 3.40541	valid_1's rmse: 3.653
Training until validation scores don't improve for 200 rounds.


...

Early stopping, best iteration is:
[1084]	training's rmse: 3.37708	valid_1's rmse: 3.54414
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[899]	training's rmse: 3.37898	valid_1's rmse: 3.64827
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1150]	training's rmse: 3.31846	valid_1's rmse: 3.7135
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[892]	training's rmse: 3.38345	valid_1's rmse: 3.64863
shape: (201917, 111)
CV 4 mean score: 3.6386, std: 0.0607, max: 3.7135, min: 3.5441.
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1101]	training's rmse: 3.37521	valid_1's rmse: 3.54493
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[870]	training's rmse: 3.38528	valid_1's rmse: 3.65145
Training until validation scores don't improve for 200 rou

shape: (201917, 111)
CV 4 mean score: 3.6395, std: 0.0602, max: 3.7139, min: 3.5458.
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1034]	training's rmse: 3.38549	valid_1's rmse: 3.54388
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[812]	training's rmse: 3.39426	valid_1's rmse: 3.64951
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1015]	training's rmse: 3.34075	valid_1's rmse: 3.71482
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[872]	training's rmse: 3.38737	valid_1's rmse: 3.64955
shape: (201917, 111)
CV 4 mean score: 3.6394, std: 0.0613, max: 3.7148, min: 3.5439.
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1484]	training's rmse: 3.31333	valid_1's rmse: 3.54463
Training until validation scores don't improve for 200 rounds.

Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1173]	training's rmse: 3.33414	valid_1's rmse: 3.64884
shape: (201917, 111)
CV 4 mean score: 3.6394, std: 0.0613, max: 3.7147, min: 3.5437.
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1165]	training's rmse: 3.36217	valid_1's rmse: 3.5444
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[959]	training's rmse: 3.36976	valid_1's rmse: 3.65055
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[976]	training's rmse: 3.34992	valid_1's rmse: 3.71442
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[881]	training's rmse: 3.38569	valid_1's rmse: 3.64917
shape: (201917, 111)
CV 4 mean score: 3.6396, std: 0.0610, max: 3.7144, min: 3.5444.
Training until validation scores don't improve for 200 rounds.
E

Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[731]	training's rmse: 3.39042	valid_1's rmse: 3.71535
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[794]	training's rmse: 3.40064	valid_1's rmse: 3.64969
shape: (201917, 111)
CV 4 mean score: 3.6401, std: 0.0605, max: 3.7154, min: 3.5463.
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1637]	training's rmse: 3.29057	valid_1's rmse: 3.54429
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1046]	training's rmse: 3.35692	valid_1's rmse: 3.64943
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1106]	training's rmse: 3.32456	valid_1's rmse: 3.71497
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1105]	training's rmse: 3.34238	valid_1's rmse: 3

Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[975]	training's rmse: 3.3666	valid_1's rmse: 3.64949
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1062]	training's rmse: 3.33172	valid_1's rmse: 3.71532
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[869]	training's rmse: 3.38721	valid_1's rmse: 3.64851
shape: (201917, 111)
CV 4 mean score: 3.6399, std: 0.0605, max: 3.7153, min: 3.5461.
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1401]	training's rmse: 3.3249	valid_1's rmse: 3.54347
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[687]	training's rmse: 3.41947	valid_1's rmse: 3.64932
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1043]	training's rmse: 3.33732	valid_1's rmse: 3.71

In [742]:
use_columns

['feature_1',
 'feature_2',
 'elapsed_time',
 'feature_1_outliers_mean',
 'feature_1_outliers_sum',
 'feature_2_outliers_mean',
 'hist_subsector_id_nunique',
 'hist_year_nunique',
 'hist_month_nunique',
 'hist_month_mean',
 'hist_month_min',
 'hist_month_max',
 'hist_hour_nunique',
 'hist_hour_min',
 'hist_hour_max',
 'hist_weekofyear_nunique',
 'hist_weekofyear_min',
 'hist_day_nunique',
 'hist_weekday_mean',
 'hist_weekday_min',
 'hist_weekday_max',
 'hist_purchase_amount_sum',
 'hist_purchase_amount_min',
 'hist_purchase_amount_mean',
 'hist_installments_sum',
 'hist_installments_max',
 'feature_3',
 'feature_2_outliers_sum',
 'hist_merchant_id_nunique',
 'hist_purchase_amount_skew',
 'hist_purchase_date_max',
 'hist_month_lag_max',
 'hist_month_lag_min',
 'hist_month_lag_mean',
 'hist_month_diff_mean',
 'hist_month_diff_std',
 'hist_authorized_flag_sum',
 'hist_authorized_flag_mean',
 'hist_authorized_flag_std',
 'hist_authorized_flag_skew',
 'hist_category_1_mean',
 'hist_category

In [773]:
use_columns

['feature_1',
 'feature_2',
 'elapsed_time',
 'feature_1_outliers_mean',
 'feature_1_outliers_sum',
 'feature_2_outliers_mean',
 'hist_subsector_id_nunique',
 'hist_year_nunique',
 'hist_month_nunique',
 'hist_month_mean',
 'hist_month_min',
 'hist_month_max',
 'hist_hour_nunique',
 'hist_hour_min',
 'hist_hour_max',
 'hist_weekofyear_nunique',
 'hist_weekofyear_min',
 'hist_day_nunique',
 'hist_weekday_mean',
 'hist_weekday_min',
 'hist_weekday_max',
 'hist_purchase_amount_sum',
 'hist_purchase_amount_min',
 'hist_purchase_amount_mean',
 'hist_installments_sum',
 'hist_installments_max',
 'feature_3',
 'feature_2_outliers_sum',
 'hist_merchant_id_nunique',
 'hist_purchase_amount_skew',
 'hist_purchase_date_max',
 'hist_month_lag_max',
 'hist_month_lag_min',
 'hist_month_lag_mean',
 'hist_month_diff_mean',
 'hist_month_diff_std',
 'hist_authorized_flag_sum',
 'hist_authorized_flag_mean',
 'hist_authorized_flag_std',
 'hist_authorized_flag_skew',
 'hist_category_1_mean',
 'hist_category

In [770]:
folds = KFold(n_splits=11, shuffle=True, random_state=SEED)

X = X_temp[use_columns]
X_test = X_temp_test[use_columns]
scores = []
prediction = np.zeros(len(X_test))

for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
    dtrain = lgb.Dataset(X.iloc[train_index], label=y.iloc[train_index])
    dvalid = lgb.Dataset(X.iloc[valid_index], label=y.iloc[valid_index])

    params = {
        'boosting': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': 0.01,
        'subsample': 0.9855232997390695,
        'max_depth': 7,
        'top_rate': 0.9064148448434349,
        'num_leaves': 63,
        'min_child_weight': 41.9612869171337,
        'other_rate': 0.0721768246018207,
        'reg_alpha': 9.677537745007898,
        'colsample_bytree': 0.5665320670155495,
        'min_split_gain': 9.820197773625843,
        'reg_lambda': 8.2532317400459,
        'min_data_in_leaf': 21,
        'verbose': -1,
        'seed':int(2**fold_n),
        'bagging_seed':int(2**fold_n),
        'drop_seed':int(2**fold_n)
    }

    model = lgb.train(
        params,
        dtrain,
        2000,          
        valid_sets=[dtrain, dvalid],
        verbose_eval=-1,
        early_stopping_rounds=200)


    y_pred_valid = model.predict(X.iloc[valid_index], num_iteration=model.best_iteration)
    scores.append(np.sqrt(mean_squared_error(y.iloc[valid_index], y_pred_valid)))

    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    prediction += y_pred / folds.n_splits   

    del model

print('shape:', X.shape)
print('CV {0:} mean score: {1:.4f}, std: {2:.4f}, max: {3:.4f}, min: {4:.4f}.'.format(4, np.mean(scores), np.std(scores), np.max(scores), np.min(scores)))

Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1283]	training's rmse: 3.3595	valid_1's rmse: 3.58537
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1257]	training's rmse: 3.36297	valid_1's rmse: 3.54179
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1482]	training's rmse: 3.33398	valid_1's rmse: 3.58547
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[876]	training's rmse: 3.4039	valid_1's rmse: 3.7821
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[919]	training's rmse: 3.42446	valid_1's rmse: 3.50307
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[773]	training's rmse: 3.42707	valid_1's rmse: 3.67807
Training until validation scores don't improve for 200 rounds.
Early stopping, bes

In [771]:
submission = pd.read_csv(os.path.join('..', 'input', 'sample_submission.csv'))
submission['target'] = prediction
submission.to_csv(os.path.join('..', 'submission', 'lightgbm_outlier_{}.csv'.format(str(datetime.datetime.today().date()).replace('-', ''))), index=False)

In [772]:
pd.DataFrame(use_columns, columns=['columns']).to_csv('use_cols2.csv', index=False)

# stacking

In [778]:
PATH = os.path.join('..', 'remove_outlier_data')

KEY = 'card_id'
SEED = 6
NTHREAD = cpu_count()
NFOLD = 11

params = {
    'num_leaves': 31,
    'min_data_in_leaf': 30, 
    'objective':'regression',
    'max_depth': -1,
    'learning_rate': 0.01,
    'boosting': 'gbdt',
    'feature_fraction': 0.6,
    'bagging_freq': 1,
    'bagging_fraction': 0.8,
    'bagging_seed': 11,
    'metric': 'rmse',
    'lambda_l1': 0.1,
    'verbosity': -1,
    'nthread': NTHREAD,
    'random_state': SEED
}

features = []

features += [f'f10{i}.pkl' for i in (2, 3, 4, 5, 6, 7, 8)]
features += [f'f11{i}_{j}.pkl' for i in (1,) 
                               for j in ('Y', 'N')]
features += [f'f13{i}.pkl' for i in (1, 3, 4, 5)]

features += [f'f20{i}.pkl' for i in (2, 3, 4, 5, 6, 7, 8)]
features += [f'f23{i}.pkl' for i in (1, 5)]

features += [f'f30{i}.pkl' for i in (2, )]

In [779]:
train = pd.read_csv(os.path.join(PATH, 'train.csv'))
test = pd.read_csv(os.path.join(PATH, 'test.csv'))

for f in tqdm(features):
    t = pd.read_pickle(os.path.join('..', 'remove_outlier_feature', f))
    train = pd.merge(train, t, on=KEY, how='left')
    test = pd.merge(test, t, on=KEY, how='left')


100%|██████████| 23/23 [00:31<00:00,  1.37s/it]


In [780]:
df = pd.concat([train, test], axis=0)
df['first_active_month'] = pd.to_datetime(df['first_active_month'])

date_features = [
    'hist_purchase_date_max','hist_purchase_date_min',
    'new_purchase_date_max', 'new_purchase_date_min',
    'Y_hist_auth_purchase_date_max', 'N_hist_auth_purchase_date_max',
    'Y_hist_auth_purchase_date_min', 'N_hist_auth_purchase_date_min'
]

for f in date_features:
    df[f] = pd.to_datetime(df[f])

df['hist_first_buy'] = (df['hist_purchase_date_min'].dt.date - df['first_active_month'].dt.date).dt.days
df['hist_last_buy'] = (df['hist_purchase_date_max'].dt.date - df['first_active_month'].dt.date).dt.days
df['new_first_buy'] = (df['new_purchase_date_min'].dt.date - df['first_active_month'].dt.date).dt.days
df['new_last_buy'] = (df['new_purchase_date_max'].dt.date - df['first_active_month'].dt.date).dt.days

for f in date_features:
    df[f] = df[f].astype(np.int64) * 1e-9

df['card_id_cnt_total'] = df['new_card_id_count'] + df['hist_card_id_count']
df['purchase_amount_total'] = df['new_purchase_amount_sum'] + df['hist_purchase_amount_sum']
df['purchase_amount_mean'] = df['new_purchase_amount_mean'] + df['hist_purchase_amount_mean']
df['purchase_amount_max'] = df['new_purchase_amount_max'] + df['hist_purchase_amount_max']
df['purchase_amount_min'] = df['new_purchase_amount_min'] + df['hist_purchase_amount_min']
df['sum_new_CLV'] = df['new_card_id_count'] * df['new_purchase_amount_sum'] / df['new_month_diff_mean']
df['sum_hist_CLV'] = df['hist_card_id_count'] * df['hist_purchase_amount_sum'] / df['hist_month_diff_mean']
df['sum_CLV_ratio'] = df['sum_new_CLV'] / df['sum_hist_CLV']

categorical_features = [
    'feature_1', 'feature_2', 'feature_3',
]

for c in categorical_features:
    df[c] = df[c].astype('category')

df['nans'] = df.isnull().sum(axis=1)

train = df[df['target'].notnull()]
test = df[df['target'].isnull()]
y = train['target']

del df
gc.collect()

1240

In [782]:
categorical_features = ['feature_1', 'feature_2', 'feature_3']
pca = PCA(n_components=1)
pca.fit(train[categorical_features])
pca_train_values = pca.transform(train[categorical_features])
pca_test_values = pca.transform(test[categorical_features])

pca_train_values = np.transpose(pca_train_values, (1, 0))
pca_test_values = np.transpose(pca_test_values, (1, 0))

for e, (pca_train, pca_test) in enumerate(zip(pca_train_values, pca_test_values)):
    train[f'pca_feature_{e}'] = pca_train
    test[f'pca_feature_{e}'] = pca_test

In [781]:
col_not_to_use = []

In [783]:
col_not_to_use += [
    'first_active_month', 'card_id', 'target', 'outliers',
]

col_to_use = [c for c in train.columns if c not in col_not_to_use]

gc.collect()

X_temp = train[col_to_use]
X_temp_test = test[col_to_use]

In [798]:
base_columns = [
    'elapsed_time',
    'feature_1',
    'feature_2',
    'feature_3',
    'feature_1_outliers_mean',
    'feature_2_outliers_mean',
    'feature_3_outliers_mean',
    'feature_1_outliers_sum',
    'feature_2_outliers_sum',
    'feature_3_outliers_sum',
    
    # histroical_transactions
    'hist_subsector_id_nunique',
    'hist_merchant_id_nunique',
    'hist_year_nunique',
    'hist_month_nunique',
    'hist_month_min',
    'hist_month_max',
    'hist_month_mean',
    'hist_hour_nunique',
    'hist_hour_min',
    'hist_hour_max',
    'hist_weekofyear_nunique',
    'hist_weekofyear_min',
    'hist_day_nunique',
    'hist_weekday_min',
    'hist_weekday_max', 
    'hist_weekday_mean',
    'hist_purchase_amount_sum',
    'hist_purchase_amount_min',
    'hist_purchase_amount_mean',
    'hist_purchase_amount_skew',
    'hist_installments_sum',
    'hist_installments_max',
    'hist_purchase_date_max',
    'hist_month_lag_max',
    'hist_month_lag_min',
    'hist_month_lag_mean',
    'hist_authorized_flag_sum',
    'hist_authorized_flag_mean',
    'hist_authorized_flag_std',
    'hist_authorized_flag_skew',
    'hist_category_1_mean',
    'hist_category_2_nunique',
    'hist_merchant_category_id_count_sum',
    'hist_merchant_id_count_sum',
    'hist_merchant_id_count_mean',
    'hist_merchant_id_count_std',
    
    # Y_historical_transactions
    'Y_hist_auth_subsector_id_nunique',
    'Y_hist_auth_merchant_id_nunique',
    'Y_hist_auth_merchant_category_id_nunique',
    'Y_hist_auth_month_nunique',
    'Y_hist_auth_day_nunique',
    'Y_hist_auth_weekday_max',
    'Y_hist_auth_purchase_amount_sum',
    'Y_hist_auth_purchase_date_min',
    'Y_hist_auth_month_lag_min',
    'Y_hist_auth_month_lag_std',
    'Y_hist_auth_category_1_mean',
    'Y_hist_auth_category_2_mean',

    # N_historical_transactions
    'N_hist_auth_subsector_id_nunique',
    'N_hist_auth_merchant_id_nunique',
    'N_hist_auth_merchant_category_id_nunique',
    
    # new_merchant_transctions
    'new_subsector_id_nunique',
    'new_merchant_id_nunique',
    'new_merchant_category_id_nunique',
    'new_year_nunique',
    'new_month_nunique',
    'new_month_mean',
    'new_month_min',
    'new_hour_mean',
    'new_hour_min',
    'new_weekofyear_mean',
    'new_weekofyear_min',
    'new_weekofyear_max',
    'new_day_mean',
    'new_day_max',
    'new_purchase_amount_sum',
    'new_purchase_amount_max',
    'new_purchase_amount_min',
    'new_installments_skew',
    'new_purchase_date_max',
    'new_purchase_date_min',
    'new_month_lag_max',
    'new_month_lag_mean',
    'new_month_lag_std',
    'new_category_1_mean',
    'new_category_2_1_mean',
    'new_category_2_2_sum',
    'new_category_2_2_mean',
]

In [799]:
columns = X_temp.columns
columns = [c for c in columns if c not in base_columns]

In [800]:
len(columns)

546

In [801]:
len(base_columns)

88

In [802]:
import random

In [ ]:
for i in range(0, 100):
    temp_columns = random.sample(columns, k=50)

    X = X_temp[base_columns + temp_columns]
    X_test = X_temp_test[base_columns + temp_columns]

    feature_importance = pd.DataFrame()
    
    folds = KFold(n_splits=4, shuffle=True, random_state=SEED)
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        dtrain = lgb.Dataset(X.iloc[train_index], label=y.iloc[train_index])
        dvalid = lgb.Dataset(X.iloc[valid_index], label=y.iloc[valid_index])

        params = {
            'boosting': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': 0.01,
            'subsample': 0.9855232997390695,
            'max_depth': 7,
            'top_rate': 0.9064148448434349,
            'num_leaves': 63,
            'min_child_weight': 41.9612869171337,
            'other_rate': 0.0721768246018207,
            'reg_alpha': 9.677537745007898,
            'colsample_bytree': 0.5665320670155495,
            'min_split_gain': 9.820197773625843,
            'reg_lambda': 8.2532317400459,
            'min_data_in_leaf': 21,
            'verbose': -1,
            'seed':int(2**fold_n),
            'bagging_seed':int(2**fold_n),
            'drop_seed':int(2**fold_n)
        }

        model = lgb.train(
            params,
            dtrain,
            2000,          
            valid_sets=[dtrain, dvalid],
            verbose_eval=-1,
            early_stopping_rounds=200)

        y_pred_valid = model.predict(X.iloc[valid_index], num_iteration=model.best_iteration)
        scores.append(np.sqrt(mean_squared_error(y.iloc[valid_index], y_pred_valid)))
         
        fold_importance = pd.DataFrame()
        fold_importance['feature'] = X.columns
        fold_importance['importance'] = np.log1p(model.feature_importance(iteration=model.best_iteration))
        fold_importance['fold'] = fold_n + 1
        feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
        
        del model
    
#     print('shape:', X.shape)
#     print('CV {0:} mean score: {1:.4f}, std: {2:.4f}, max: {3:.4f}, min: {4:.4f}.'.format(4, np.mean(scores), np.std(scores), np.max(scores), np.min(scores)))
    
    # ==========================================================================
    # bugging
    # ==========================================================================
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))

    scores = []
    feature_importance['importance'] /= NFOLD
    bagging_columns = feature_importance[['feature', 'importance']].groupby('feature').mean().reset_index().sort_values(by='importance', ascending=False).query('importance > 0.3')['feature'].values
    
    X = X_temp[bagging_columns]
    X_test = X_temp_test[bagging_columns]
    
    folds = KFold(n_splits=11, shuffle=True, random_state=SEED)
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        dtrain = lgb.Dataset(X.iloc[train_index], label=y.iloc[train_index])
        dvalid = lgb.Dataset(X.iloc[valid_index], label=y.iloc[valid_index])

        params = {
            'boosting': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': 0.01,
            'subsample': 0.9855232997390695,
            'max_depth': 7,
            'top_rate': 0.9064148448434349,
            'num_leaves': 63,
            'min_child_weight': 41.9612869171337,
            'other_rate': 0.0721768246018207,
            'reg_alpha': 9.677537745007898,
            'colsample_bytree': 0.5665320670155495,
            'min_split_gain': 9.820197773625843,
            'reg_lambda': 8.2532317400459,
            'min_data_in_leaf': 21,
            'verbose': -1,
            'seed':int(2**fold_n),
            'bagging_seed':int(2**fold_n),
            'drop_seed':int(2**fold_n)
        }

        model = lgb.train(
            params,
            dtrain,
            2000,          
            valid_sets=[dtrain, dvalid],
            verbose_eval=-1,
            early_stopping_rounds=200)

        y_pred_valid = model.predict(X.iloc[valid_index], num_iteration=model.best_iteration)
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(np.sqrt(mean_squared_error(y.iloc[valid_index], y_pred_valid)))

        y_pred = model.predict(X_test, num_iteration=model.best_iteration)
        prediction += y_pred / folds.n_splits   

        del model

#     print('shape:', X.shape)
#     print('CV {0:} mean score: {1:.4f}, std: {2:.4f}, max: {3:.4f}, min: {4:.4f}.'.format(11, np.mean(scores), np.std(scores), np.max(scores), np.min(scores)))
    
    np.save(os.path.join('stacking', 'oof', '{0:d}_oof_{1:d}_lgb'.format(i, int(np.mean(scores) * 10000))), oof)
    np.save(os.path.join('stacking', 'pred', '{0:d}_pred_lgb'.format(i)), prediction)

Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1118]	training's rmse: 3.36995	valid_1's rmse: 3.55371
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[894]	training's rmse: 3.38083	valid_1's rmse: 3.65661
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1088]	training's rmse: 3.33208	valid_1's rmse: 3.72038
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[898]	training's rmse: 3.38573	valid_1's rmse: 3.66304
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1380]	training's rmse: 3.3396	valid_1's rmse: 3.59548
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1603]	training's rmse: 3.30873	valid_1's rmse: 3.54724
Training until validation scores don't improve for 200 rounds.
Early stopping, 

Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[887]	training's rmse: 3.41712	valid_1's rmse: 3.67652
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[718]	training's rmse: 3.43594	valid_1's rmse: 3.74726
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[728]	training's rmse: 3.45511	valid_1's rmse: 3.57664
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[721]	training's rmse: 3.45247	valid_1's rmse: 3.62698
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1304]	training's rmse: 3.35664	valid_1's rmse: 3.71269
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[688]	training's rmse: 3.44299	valid_1's rmse: 3.71978
Training until validation scores don't improve for 200 rounds.
Early stopping, be